## ALS

## 0. Setup

In [1]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

# Manuseio nos dados

from os import chdir

# ALS

from implicit import als

/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#---- Mudando o diretório raiz para a pasta principal do projeto

chdir('../../')

#---- Template dos gráficos:

pio.templates.default = "plotly_white"

#---- Carregando as funções

from functions import *

## 1. Extração dos dados

In [3]:
#---- Dados de treino

train_data = load_sparse_csr('03-data/02-processed/04-ALS-train-data.npz')

train_data

<22568x158235 sparse matrix of type '<class 'numpy.int64'>'
	with 346011 stored elements in Compressed Sparse Row format>

In [4]:
#---- Dados de teste

test_data = pd.read_parquet('03-data/02-processed/02-test_data.parquet')

test_data.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22568 entries, 0 to 22567
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   User-ID              22568 non-null  int64  
 1   ISBN                 22568 non-null  object 
 2   Book-Rating          22568 non-null  int64  
 3   Book-Title           18886 non-null  object 
 4   Book-Author          18886 non-null  object 
 5   Year-Of-Publication  18886 non-null  object 
 6   Publisher            18886 non-null  object 
 7   Age                  15357 non-null  float64
 8   city                 22568 non-null  object 
 9   state                22568 non-null  object 
 10  country              22568 non-null  object 
 11  RN                   22568 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.1+ MB


In [6]:
#---- De-para's:

de_para_isbn = pd.read_parquet('03-data/02-processed/05-ALS-DE-PARA-ISBN.parquet')
de_para_user = pd.read_parquet('03-data/02-processed/06-ALS-DE-PARA-USER.parquet')

## 2. Modelagem

In [7]:
model = als.AlternatingLeastSquares(factors = 50)

model

/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [8]:
model.fit(train_data)

100%|███████████████████████████████████████████| 15/15 [00:35<00:00,  2.38s/it]


In [10]:
user_id_rec = 22406

de_para_user_id_rec = de_para_user.query(f'`User-ID` == {user_id_rec}')['INDEX'].iloc[0]
de_para_user_id_rec

2

In [11]:
#---- Recomendação para um usuário:

recommendations = model.similar_users(userid = de_para_user_id_rec, N = 10)

recommendations

(array([    2, 14342, 15528,   665, 19734,  3827, 18639, 16812, 21281,
         8393], dtype=int32),
 array([1.0000001 , 0.84841084, 0.59876746, 0.5863348 , 0.58539367,
        0.5799726 , 0.56359124, 0.5616552 , 0.5486857 , 0.5469215 ],
       dtype=float32))

In [12]:
isnb_rec = 47405
de_para_isbn_rec = de_para_isbn.query(f'`ISBN` == {isnb_rec}')['INDEX'].iloc[0]
de_para_isbn_rec

158230

In [13]:
#---- Recomendação para um item:

recommendations = model.similar_items(itemid = de_para_isbn_rec)

recommendations

(array([74161,  2030,  1907,  1614,  1491,  1356,  1347,  1193,   424,
          243], dtype=int32),
 array([1.0000001, 1.0000001, 1.0000001, 1.0000001, 1.0000001, 1.0000001,
        1.0000001, 1.0000001, 1.0000001, 1.0000001], dtype=float32))

# TO-DO's

- Ler um pouco mais sobre a teoria do ALS
- Criar um código que exclua o primeiro item da recomendação do usuário